In [11]:
import requests

url = "http://localhost:4000/key/info"
key = 'sk-7vj9bFDk30w6h2TFS31DQw'
auth_token = "sk-1234"

# Constructing the headers
headers = {
    "Authorization": f"Bearer {auth_token}"
}

# Sending the GET request
response = requests.get(url, params={"key": key}, headers=headers)

# Printing the response
print(response.status_code)
print(response.json())

200
{'key': 'sk-7vj9bFDk30w6h2TFS31DQw', 'info': {'key_name': 'sk-...1DQw', 'key_alias': 'Amzo', 'soft_budget_cooldown': False, 'spend': 0.0, 'expires': None, 'models': [], 'aliases': {}, 'config': {}, 'user_id': None, 'team_id': None, 'permissions': {}, 'max_parallel_requests': None, 'metadata': {'team': 'AiTeam', 'user': 'Amzo', 'email': 'ag@gmail.com'}, 'blocked': None, 'tpm_limit': None, 'rpm_limit': None, 'max_budget': 10.0, 'budget_duration': None, 'budget_reset_at': None, 'allowed_cache_controls': [], 'model_spend': {}, 'model_max_budget': {}, 'budget_id': None, 'created_at': '2024-12-19T20:58:26.469000+00:00', 'updated_at': '2024-12-19T20:58:26.469000+00:00', 'litellm_budget_table': None}}


In [25]:
response.json()

{'key': 'sk-8nuLzOpAdzLjKUGh1ziMvQ',
 'info': {'key_name': 'sk-...iMvQ',
  'key_alias': 'Gadji',
  'soft_budget_cooldown': False,
  'spend': 1.836017,
  'expires': None,
  'models': [],
  'aliases': {},
  'config': {},
  'user_id': None,
  'team_id': None,
  'permissions': {},
  'max_parallel_requests': None,
  'metadata': {'team': 'AiTeam', 'user': 'Gadji', 'email': 'ag@gmail.com'},
  'blocked': None,
  'tpm_limit': None,
  'rpm_limit': None,
  'max_budget': 10.0,
  'budget_duration': None,
  'budget_reset_at': None,
  'allowed_cache_controls': [],
  'model_spend': {},
  'model_max_budget': {},
  'budget_id': None,
  'created_at': '2024-12-19T16:49:13.083000+00:00',
  'updated_at': '2024-12-19T16:51:25.571000+00:00',
  'litellm_budget_table': None}}

In [19]:
!curl -X POST 'http://0.0.0.0:4000/team/new'\
    -H 'Authorization: Bearer sk-1234'\
    -H 'Content-Type: application/json'\
    -d '{"team_alias": "Sales", \
        "max_budget": 10, "max_parallel_requests": 10, \
        "organization_id":"16f0cb2e-1e2b-4b38-9623-bfc754efb6ad"}' 

Internal Server Error

In [3]:

def hash_token(token: str):
    import hashlib

    # Hash the string using SHA-256
    hashed_token = hashlib.sha256(token.encode()).hexdigest()

    return hashed_token

In [8]:
hash_token(token=key)

'70bf186f10a2b45e8f7e0973ef0c5a5fa2ded887447b318714ea77d0a2d70fe3'

In [9]:
psql -U admin -d keys_db
psql
\dt # list database
SELECT * FROM "LiteLLM_SpendLogs";

SyntaxError: invalid syntax (207769578.py, line 1)

```prisma
// View spend, model, api_key per request
model LiteLLM_SpendLogs {
  request_id          String @id
  call_type           String
  api_key             String  @default ("") // Hashed API Token. Not the actual Virtual Key. Equivalent to 'token' column in LiteLLM_VerificationToken
  spend               Float    @default(0.0)
  total_tokens        Int     @default(0)
  prompt_tokens       Int     @default(0)
  completion_tokens   Int     @default(0)
  startTime           DateTime // Assuming start_time is a DateTime field
  endTime             DateTime // Assuming end_time is a DateTime field
  completionStartTime DateTime? // Assuming completionStartTime is a DateTime field
  model               String   @default("")
  model_id            String?   @default("") // the model id stored in proxy model db
  model_group         String?   @default("") // public model_name / model_group
  custom_llm_provider String?   @default("") // litellm used custom_llm_provider
  api_base            String?   @default("")
  user                String?   @default("")
  metadata            Json?     @default("{}")
  cache_hit           String?   @default("")
  cache_key           String?   @default("")
  request_tags        Json?     @default("[]")
  team_id             String? 
  end_user            String?
  requester_ip_address String?
  @@index([startTime])
  @@index([end_user])
}
```

### SQL Code

In [29]:
sql_query = """
WITH SpendByModelApiKey AS (
    SELECT
        sl.api_key,
        sl.model,
        SUM(sl.spend) AS model_cost,
        SUM(sl.prompt_tokens) AS model_input_tokens,
        SUM(sl.completion_tokens) AS model_output_tokens
    FROM
        "LiteLLM_SpendLogs" sl
    WHERE
        sl."startTime" BETWEEN $1::date AND $2::date AND sl.api_key = $3
    GROUP BY
        sl.api_key,
        sl.model
)
SELECT
    api_key,
    SUM(model_cost) AS total_cost,
    SUM(model_input_tokens) AS total_input_tokens,
    SUM(model_output_tokens) AS total_output_tokens,
    jsonb_agg(jsonb_build_object(
        'model', model,
        'total_cost', model_cost,
        'total_input_tokens', model_input_tokens,
        'total_output_tokens', model_output_tokens
    )) AS model_details
FROM
    SpendByModelApiKey
GROUP BY
    api_key
ORDER BY
    total_cost DESC;
"""

In [30]:
db_host = 'postgres'
db_port = '5432'
db_user = 'admin'
db_name = 'keys_db'
password = 'secret'

db_url = f"postgresql://{db_user}:{password}@{db_host}:{db_port}/{db_name}"

In [6]:
from sqlalchemy import create_engine

engine = create_engine(db_url)

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
db_response = await prisma_client.db.query_raw(
sql_query, start_date_obj, end_date_obj, api_key
)